In [ ]:
from binary_models.load_ternary import load_community_model
community, model_file = load_community_model()

## Optimize Model

In [4]:
result = community.optimize(fluxes=True, pfba=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,130.294226,2492,2024
Rhodosporidium,0.333333,33.144143,2423,2076
Synechococcus,0.333333,1.999997,968,884
medium,NaN,NaN,466,466


### Update Medium

In [8]:
# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_sucr_m"] = 0
medium_to_change["EX_glc__D_m"] = 0
medium_to_change["EX_leu__L_m"] = 0
#medium_to_change["EX_glyc__R_m"] = 0
medium_to_change["EX_nh4_m"] = 0
medium_to_change["EX_no3_m"] = 0
medium_to_change["EX_n2_m"] = 5


# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

{'EX_co2_m': 1.99,
 'EX_h_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_pi_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_4abut_m': 1000.0,
 'EX_ac_m': 1000.0,
 'EX_acgam_m': 999999.0,
 'EX_ade_m': 1000.0,
 'EX_adn_m': 1000.0,
 'EX_ala__D_m': 1000.0,
 'EX_ala__L_m': 1000.0,
 'EX_alltn_m': 1000.0,
 'EX_arg__L_m': 999999.0,
 'EX_asn__L_m': 1000.0,
 'EX_asp__L_m': 1000.0,
 'EX_ca2_m': 999999.0,
 'EX_cl_m': 999999.0,
 'EX_csn_m': 1000.0,
 'EX_cys__L_m': 999999.0,
 'EX_cytd_m': 1000.0,
 'EX_dad_2_m': 1000.0,
 'EX_dca_m': 1000.0,
 'EX_fe2_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_for_m': 1000.0,
 'EX_fru_m': 1000.0,
 'EX_gal_m': 1000.0,
 'EX_galur_m': 1000.0,
 'EX_gam_m': 999999.0,
 'EX_glu__L_m': 999999.0,
 'EX_gly_m': 999999.0,
 'EX_glyc_m': 1000.0,
 'EX_gua_m': 1000.0,
 'EX_hco3_m': 1.99,
 'EX_his__L_m': 1000.0,
 'EX_ile__L_m': 999999.0,
 'EX_inost_m': 1000.0,
 'EX_ins_m': 1000.0,
 'EX_k_m': 999999.0,
 'EX_lac__L_m': 1000.0,
 'EX_lys__L_m': 1000.0,
 'EX_malt_m': 1000.0,
 'EX_

In [9]:
result_altered_medium = community.optimize(fluxes=True, pfba=True)

In [11]:
result_altered_medium

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,130.118082,2492,2024
Rhodosporidium,0.333333,30.523491,2423,2076
Synechococcus,0.333333,1.999997,968,884
medium,NaN,NaN,466,466
